# Exploring OOI data access

Still not sure how everything works, so this script is going to be my exploration into loading broadband hydrophone data.

I'm using this notebook as an example: https://github.com/petercable/noisy-dolphins/blob/master/Voices.ipynb 

But rather than loading up one specific file, I'm interested in providing the user with a way to explore the available data a little bit. I plan to use beautiful soup to scrape the particular corner of the OOI raw data website that I'm interested in. It may not be the best way to do it but I currently don't know a better way so here we go!

In [10]:
import obspy
import requests
from bs4 import BeautifulSoup


## Setting up and scraping data

I'm starting by specifying the URL to the page I'm interested in. (I've also specified a particular miniseed file but commenting that out for now).

In [8]:
# url_mseed = 'https://rawdata.oceanobservatories.org/files/RS03AXBS/LJ03A/09-HYDBBA302/2020/12/05/OO-AXVM1--YDH-2020-12-05T14:30:00.000015.mseed'
# print('The requested URL: ' + url_mseed)

page_url = 'https://rawdata.oceanobservatories.org/files/RS03AXBS/LJ03A/09-HYDBBA302/2020/12/05/'
page = requests.get(page_url).content

Now I'll use beautiful soup to parse the contents of the page.

In [38]:
soup = BeautifulSoup(page, 'html.parser')
# Uncomment to see the full html
# print(soup.prettify()) 
# pull out every link on the page
all_links = [link.get('href') for link in soup.find_all('a')]
mseed_files = [i for i in all_links if '.mseed' in i]